# Transcriptional trajectories in mouse limb buds reveal the transition from anterior-posterior to proximal-distal patterning at early limb bud stage.
Desanlis I, Paul R, Kmita M. J Dev Biol. 2020;8(4):1–16. 
 
 PMID: [7768367](http://www.ncbi.nlm.nih.gov/pmc/articles/pmc7768367/)
 ***
 

## 0. Setting up workenvironment<a id="0"></a>

In [ ]:
suppressPackageStartupMessages({
    library(DropletUtils)
    library(SingleCellExperiment)
    library(scuttle)
    library(Seurat)
    library(SeuratWrappers)
    library(stringr)
    library(dplyr)
    library(data.table)
    library(Matrix)
    library(patchwork)
    library(ggplot2)
})

options(repr.plot.width = 16, repr.plot.height = 8)

## 3. E10.5<a id="8"></a>

### 3.1. Data analysis Seurat<a id="9"></a>

In [ ]:
E10_5 <- readRDS(file = "/home/host_home/tom/Kmita/6-kmita/RDS_QC/E10_5.Rds")

In [ ]:
VlnPlot(E10_5, features = c("nCount_RNA", "nFeature_RNA", "subsets_Mito_percent"), 
        ncol = 3, group.by = "orig.ident", pt.size = 0)

In [ ]:
FeatureScatter(E10_5, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")

#### 3.1.1. Feature selection

We next calculate a subset of features that exhibit high cell-to-cell variation in the dataset (i.e, they are highly expressed in some cells, and lowly expressed in others). We and others have found that focusing on these genes in downstream analysis helps to highlight biological signal in single-cell datasets.

The procedure in Seurat v3 directly models the mean-variance relationship inherent in single-cell data, and is implemented in the `FindVariableFeatures` function. By default, it defaults to 2,000 features (genes) per dataset. These will be used in downstream analysis, like principal component analysis (PCA).

In [ ]:
E10_5 <- NormalizeData(E10_5, verbose = FALSE)
E10_5 <- FindVariableFeatures(E10_5, selection.method = "vst", nfeatures = 2000)

# Identify the 10 most highly variable genes
top10 <- head(VariableFeatures(E10_5), 10)

In [ ]:
E10_5

In [ ]:
FeatureScatter(E10_5, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")

In [ ]:
E10_5 <- NormalizeData(E10_5, verbose = FALSE)
E10_5 <- FindVariableFeatures(E10_5, selection.method = "vst", nfeatures = 5000)

# Identify the 10 most highly variable genes
top10 <- head(VariableFeatures(E10_5), 10)

In [ ]:
E10_5

In [ ]:
options(repr.plot.width=10)

# plot variable features with and without labels
plot1 <- VariableFeaturePlot(E10_5)
plot2 <- LabelPoints(plot = plot1, points = top10, repel = TRUE)
plot2

#### 3.1.2. Scaling the data

Next, we apply a linear transformation (scaling) that is a standard pre-processing step prior to dimensional reduction techniques like PCA. The ScaleData function:

- Shifts the expression of each gene, so that the mean expression across cells is 0
- Scales the expression of each gene, so that the variance across cells is 1
- This step gives equal weight in downstream analyses, so that highly-expressed genes do not dominate. This is superior to log-normalized counts, which overemphasize the influence of small count fluctuations.
- The results of this are stored in `object[["originalexp"]]@scale.data`

In [ ]:
all.genes <- rownames(E10_5)
E10_5 <- ScaleData(E10_5, features = all.genes)

#### 3.1.3. Linear dimensionality reduction: PCA

Next we perform PCA on the scaled data. By default, only the previously determined variable features are used as input, but can be defined using features argument if you wish to choose a different subset.

In [ ]:
E10_5 <- RunPCA(E10_5, features = VariableFeatures(object = E10_5), verbose = FALSE)

In [ ]:
DimPlot(E10_5, reduction = "pca", group.by = "Sample_Name") + xlab("PC 1") + ylab("PC 2") 

##### **3.1.3.1. Determine dimensionality**

To overcome the extensive technical noise in any single feature for scRNA-seq data, Seurat clusters cells based on their PCA scores, with each PC essentially representing a 'metafeature' that combines information across a correlated feature set. The top principal components therefore represent a robust compression of the dataset. However, how many componenets should we choose to include? 

[Macosko et al](http://www.cell.com/abstract/S0092-8674(15)00549-8) introduced a resampling test inspired by the JackStraw procedure. They randomly permute a subset of the data (1% by default) and rerun PCA, constructing a 'null distribution' of feature scores, and repeat this procedure. 'Significant' PCs are defined as those who have a strong enrichment of low p-value features.

**NOTE: This process can take a long time for big datasets, and is therefore commented out for expediency. More approximate techniques such as a scree plot implemented in `ElbowPlot()` can be used to reduce computation time.**

The `JackStrawPlot` function provides a visualization tool for comparing the distribution of p-values for each PC with a uniform distribution (dashed line). 'Significant' PCs will show a strong enrichment of features with low p-values (solid curve above the dashed line).

In [ ]:
# object <- JackStraw(object, num.replicate = 100)
# object <- ScoreJackStraw(object, dims = 1:20)

In [ ]:
# JackStrawPlot(object, dims = 1:15)

An alternative heuristic method generates an 'Elbow plot': a ranking of principle components based on the percentage of variance explained by each one (ElbowPlot function). 

In [ ]:
ElbowPlot(E10_5, 50)

### 3.2. Clustering<a id="10"></a>

Seurat v4 applies a graph-based clustering approach, building upon initial strategies in [Macosko et al](http://www.cell.com/abstract/S0092-8674(15)00549-8). Importantly, the distance metric which drives the clustering analysis (based on previously identified PCs) remains the same. The new method embeds cells in a graph structure - for example a K-nearest neighbor (KNN) graph, with edges drawn between cells with similar feature expression patterns, and then attempt to partition this graph into highly interconnected 'quasi-cliques' or 'communities'.

As in [PhenoGraph](https://doi.org/10.1016/j.cell.2015.05.047), Seurat first constructs a KNN graph based on the euclidean distance in PCA space, and refine the edge weights between any two cells based on the shared overlap in their local neighborhoods (Jaccard similarity). This step is performed using the `FindNeighbors` function, and takes as input the previously defined dimensionality of the dataset.

To cluster the cells, modularity optimization techniques such as the Louvain algorithm are then applied to iteratively group cells together, with the goal of optimizing the standard modularity function. The `FindClusters` function implements this procedure and contains a resolution parameter that sets the 'granularity' of the downstream clustering, with increased values leading to a greater number of clusters. Optimal resolution often increases for larger datasets.

In [ ]:
E10_5 <- FindNeighbors(E10_5, dims = 1:35)
E10_5 <- FindClusters(E10_5, resolution = 0.065)

### 3.3. Non-linear dimensionality reduction: UMAP <a id="11"></a>

Seurat offers several non-linear dimensional reduction techniques, such as tSNE and UMAP, to visualize and explore these datasets. The goal of these algorithms is to learn the underlying manifold of the data in order to place similar cells together in low-dimensional space. Cells within the graph-based clusters determined above should co-localize on these dimension reduction plots. As input to the UMAP and tSNE, [it is recommended to use the same PCs](https://doi.org/10.1038/s41467-019-13056-x) as input to the clustering analysis.

In [ ]:
E10_5 <- RunUMAP(E10_5, dims = 1:35, n.neighbors = 12, verbose = FALSE)

In [ ]:
options(repr.plot.width = 8)

DimPlot(E10_5, reduction = "umap", label = TRUE) + xlab("UMAP 1") + ylab("UMAP 2")
#ggsave("./6-kmita/images/no_annotation_E10_5.png" ,width=12, height=8)

In [ ]:
#E10_5 <- FindNeighbors(E10_5, dims = 1:35)
#E10_5 <- FindClusters(E10_5, resolution = 0.5)

#E10_5 <- RunUMAP(E10_5, dims = 1:35, n.neighbors = 12, verbose = FALSE)

#DimPlot(E10_5, reduction = "umap", label = TRUE) + xlab("UMAP 1") + ylab("UMAP 2")

### 3.4. Differential expression analysis<a id="12"></a>
#### 3.4.1. Global annotation by differential expression analysis<a id="13"></a>

Seurat identifies markers that define clusters via differential expression. By default, it identifes positive and negative markers of a single cluster (specified in ident.1), compared to all other cells. `FindAllMarkers` automates this process for all clusters, but it is possible to test groups of clusters vs. each other, or against all cells.

The `min.pct` argument requires a feature to be detected at a minimum percentage in either of the two groups of cells, and the `thresh.test` argument requires a feature to be differentially expressed (on average) by some amount between the two groups. You can set both of these to 0, but with a dramatic increase in time - since this will test a large number of features that are unlikely to be highly discriminatory. As another option to speed up these computations, `max.cells.per.ident` can be set. This will downsample each identity class to have no more cells than whatever this is set to. While there is generally going to be a loss in power, the speed increases can be significiant and the most highly differentially expressed features will likely still rise to the top.

In [ ]:
# Seurat default settings
E10.5.markers <- FindAllMarkers(E10_5, only.pos = TRUE, min.pct = 0.25, logfc.threshold = 0.25)

In [ ]:
#dir.create("./6-kmita/analysis")
write.csv(E10.5.markers, file = "./6-kmita/analysis/E10.5.markers.csv", quote = FALSE)

Cluster 0: mesenchymal cells 

In [ ]:
#CD73, CD90, CD105, CD106, CD146, STRO-1

options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(E10_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(E10_5, c("Prrx1", "Tbx4", "Tbx5", "Prrx2", "Pitx1", "Hoxd9"), min.cutoff = "q1")  

p1 + p2

#ggsave("./6-kmita/images/E10_5_c0.png" ,width=18, height=12)

In [ ]:
#CD73, CD90, CD105, CD106, CD146, STRO-1

options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(E10_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(E10_5, c("Prrx1", "Tbx4", "Tbx5", "Prrx2", "Shox2", "Pitx1"), min.cutoff = "q1")  

p1 + p2

#ggsave("./6-kmita/images/E10_5_c0.png" ,width=18, height=12)

Cluster 1: ectoderm

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 8)

p1 <- DimPlot(E10_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(E10_5, c("Wnt3", "Fgf8", "En1", "Wnt7a"), min.cutoff = "q1") 

p1 + p2
#ggsave("./6-kmita/images/E10_5_c1.png" ,width=16, height=8)

#Test: En1 -> engrailed 1 -> responseble for vertralisation -> target gene BMPs 

Cluster 2: muscle cells

In [ ]:
options(repr.plot.width = 16)

p1 <- DimPlot(E10_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(E10_5, c("Pax7", "Myf5", "Pax3", "Myod1"), min.cutoff = "q1") + xlab("UMAP 1") + ylab("UMAP 2")

p1 + p2
#ggsave("./6-kmita/images/E10_5_c2.png" ,width=16, height=8)

Cluster 3: endothelial cells  

In [ ]:
p1 <- DimPlot(E10_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(E10_5, c("Pecam1", "Cd34", "Cd93", "Cdh5"), min.cutoff = "q1") 

p1 + p2
#ggsave("./6-kmita/images/E10_5_c3.png" ,width=16, height=8)

Cluster 4: erythrocytes

In [ ]:
p1 <- DimPlot(E10_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(E10_5, c("Hbb-y", "Hba-x", "Hba-a1", "Hbb-bt"), min.cutoff = "q1") + xlab("UMAP 1") + ylab("UMAP 2")

p1 + p2
#ggsave("./6-kmita/images/E10_5_c4.png" ,width=16, height=8)

**Adding cell types**

In [ ]:
#creating column that contains cell type 
E10_5@meta.data$CellType <- Idents(E10_5)

In [ ]:
E10_5@meta.data$CellType <- as.character(E10_5@meta.data$CellType)

E10_5@meta.data$CellType[E10_5@meta.data$CellType == '0'] <- 'Mesenchyme'
E10_5@meta.data$CellType[E10_5@meta.data$CellType == '1'] <- 'Ectoderm'
E10_5@meta.data$CellType[E10_5@meta.data$CellType == '2'] <- 'Muscle progenitors'
E10_5@meta.data$CellType[E10_5@meta.data$CellType == '3'] <- 'Vascular endothelial cells'
E10_5@meta.data$CellType[E10_5@meta.data$CellType == '4'] <- 'Erythroid cells'

E10_5@meta.data$CellType <- as.factor(E10_5@meta.data$CellType)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)

DimPlot(E10_5, reduction = "umap", label = FALSE, group.by = "CellType") + xlab("UMAP 1") + ylab("UMAP 2")
#ggsave("./6-kmita/figures_thesis/E10_5_global_annotated.png" ,width=10, height=8)

In [ ]:
saveRDS(E10_5, file = "/home/host_home/tom/Kmita/6-kmita/RDataSessions/E10_5_GlobalAnnotation_SeuratObject.Rds")

**Visualizing data**

In [ ]:
seuratObject <- readRDS("/home/host_home/usb-drive/Thesis/Kmita/6-kmita/RDataSessions/E10_5_GlobalAnnotation_SeuratObject.Rds")

In [ ]:
E10_5@meta.data$CellType_v <- E10_5@meta.data$CellType

seuratObject <- E10_5

seuratObject@meta.data$CellType_v <- as.character(seuratObject@meta.data$CellType_v)

seuratObject@meta.data$CellType_v[seuratObject@meta.data$CellType_v == 'Mesenchyme'] <- '1 - Mesenchyme'
seuratObject@meta.data$CellType_v[seuratObject@meta.data$CellType_v == 'Ectoderm'] <- '2 - Ectoderm'
seuratObject@meta.data$CellType_v[seuratObject@meta.data$CellType_v == 'Myocytes'] <- '3 - Myocytes'
seuratObject@meta.data$CellType_v[seuratObject@meta.data$CellType_v == 'Endothelium'] <- '4 - Endothelium'
seuratObject@meta.data$CellType_v[seuratObject@meta.data$CellType_v == 'Erythrocytes'] <- '5 - Erythrocytes'

seuratObject@meta.data$CellType_v <- as.factor(seuratObject@meta.data$CellType_v)

E10_5 <- seuratObject 

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)

DimPlot(E10_5, reduction = "umap",group.by = "CellType_v",) + xlab("UMAP 1") + ylab("UMAP 2")
#ggsave("./6-kmita/figures_thesis/E10_5_global_annotated.pdf" ,width=10, height=8)

In [ ]:
features <- c("Prrx1","Tbx5","Fgf8","Wnt7a","Pax3","Myf5","Pecam1", "Cd34", "Hba-a1","Hbb-y")

In [ ]:
# Dot plots - the size of the dot corresponds to the percentage of cells expressing the
# feature in each cluster. The color represents the average expression level
DotPlot(seuratObject, features = features,col.min = 0) + RotatedAxis()

#### 3.4.2. Refinement of global annotations<a id="13.2"></a>
##### 3.4.2.1. Subsetting mesenchymal clusters<a id="14"></a>

In [ ]:
E10_5 <- readRDS("/home/host_home/tom/Kmita/6-kmita/RDataSessions/E10_5_GlobalAnnotation_SeuratObject.Rds")

In [ ]:
Idents(E10_5) <- E10_5@meta.data$CellType

In [ ]:
sc_mesenchymal_E10_5 <- subset(E10_5, idents = c("Mesenchyme"))

In [ ]:
sc_mesenchymal_E10_5 <- NormalizeData(sc_mesenchymal_E10_5, verbose = FALSE)
sc_mesenchymal_E10_5 <- FindVariableFeatures(sc_mesenchymal_E10_5, selection.method = "vst", nfeatures = 2000)

# Identify the 10 most highly variable genes
top10 <- head(VariableFeatures(sc_mesenchymal_E10_5), 10)

In [ ]:
sc_mesenchymal_E10_5

In [ ]:
sc_mesenchymal_E10_5 <- ScaleData(sc_mesenchymal_E10_5)

In [ ]:
sc_mesenchymal_E10_5 <- RunPCA(sc_mesenchymal_E10_5, features = VariableFeatures(object = sc_mesenchymal_E10_5), verbose = FALSE)

In [ ]:
options(repr.plot.width=10)
DimPlot(sc_mesenchymal_E10_5, reduction = "pca", group.by = "orig.ident") + xlab("PC 1") + ylab("PC 2")

In [ ]:
ElbowPlot(sc_mesenchymal_E10_5, 50)

In [ ]:
sc_mesenchymal_E10_5 <- FindNeighbors(sc_mesenchymal_E10_5, dims = 1:25, verbose = FALSE)
sc_mesenchymal_E10_5 <- FindClusters(sc_mesenchymal_E10_5, resolution = 2, verbose = FALSE)

In [ ]:
sc_mesenchymal_E10_5 <- RunUMAP(sc_mesenchymal_E10_5, dims = 1:25, n.neighbors = 10, verbose = FALSE)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)
DimPlot(sc_mesenchymal_E10_5, reduction = "umap", label=TRUE) + xlab("UMAP 1") + ylab("UMAP 2")

#ggsave("./6-kmita/images/E10_5_m_no_annotation.png" ,width=10, height=8)

In [ ]:
markers_10_5_mesenchyme <- FindAllMarkers(sc_mesenchymal_E10_5)

In [ ]:
#saveRDS(markers_10_5_mesenchyme, file = "/home/host_home/usb-drive/Thesis/Kmita/6-kmita/analysis/markers_10_5_mesenchyme.Rds")

In [ ]:
markers_10_5_mesenchyme <- readRDS(file = "/home/host_home/usb-drive/Thesis/Kmita/6-kmita/analysis/markers_10_5_mesenchyme.Rds")

In [ ]:
# find markers for every cluster compared to all remaining cells, report only the positive
# ones
#df_m %>%
#    group_by(cluster) %>%
#    slice_max(n = 10, order_by = avg_log2FC)

In [ ]:
#select only cluster 0 and order avg_log2FC from big to small

#df_m[df_m$cluster == 0,] %>%
#slice_max(n = 10
#          , order_by = avg_log2FC)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)
DimPlot(sc_mesenchymal_E10_5, reduction = "umap", label=TRUE) + xlab("UMAP 1") + ylab("UMAP 2")

**Positional information**

Proximal mesenchyme

In [ ]:
g <- WhichCells(sc_mesenchymal_E10_5, idents = c("22"))
p1 <- DimPlot(sc_mesenchymal_E10_5, cells.highlight = list(g)) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E10_5, c("Meis1", "Hoxa9","Shox2", "Hoxd9"),  min.cutoff = "q1") + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 20, repr.plot.height = 8)
p1 + p2

In [ ]:
g <- WhichCells(sc_mesenchymal_E10_5, idents = c("2","13","23","13","21","0","17","8",'12','10','24','20','7','9','5','11'))
p1 <- DimPlot(sc_mesenchymal_E10_5, cells.highlight = list(g), label=TRUE) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E10_5, c("Meis1", "Meis2", "Hoxd9", "Shox2" ), min.cutoff = "q1") + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 20, repr.plot.height = 8)
p1 + p2

#ggsave("./6-kmita/images/E10_5_m_proximal.png" ,width=20, height=8)

#add Hox

Intermediate mesenchyme 

In [ ]:
g <- WhichCells(sc_mesenchymal_E10_5, idents = c("15"))
p1 <- DimPlot(sc_mesenchymal_E10_5, cells.highlight = list(g)) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E10_5, c("Hoxa11")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 20, repr.plot.height = 8)
p1 + p2

In [ ]:
g <- WhichCells(sc_mesenchymal_E10_5, idents = c("2","13","23",'4','5','16','4','9','14','18','1','22','25','3'))
p1 <- DimPlot(sc_mesenchymal_E10_5, cells.highlight = list(g), label=TRUE) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E10_5, c("Hoxa11")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 20, repr.plot.height = 8)
p1 + p2

#ggsave("./6-kmita/images/E10_5_m_intermediate.png" ,width=20, height=8)

Distal mesenchyme

In [ ]:
g <- WhichCells(sc_mesenchymal_E10_5, idents = c("12"))
p1 <- DimPlot(sc_mesenchymal_E10_5, cells.highlight = list(g)) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E10_5, c("Hoxd13", "Hoxa13")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 9.5, repr.plot.height = 8)
p1 
options(repr.plot.width = 20, repr.plot.height = 8)
p2

In [ ]:
g <- WhichCells(sc_mesenchymal_E10_5, idents = c("6","15",'16','19','22'))
p1 <- DimPlot(sc_mesenchymal_E10_5, cells.highlight = list(g), label=TRUE) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E10_5, c("Hoxd13", "Hoxa13")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 9.5, repr.plot.height = 8)
p1 
options(repr.plot.width = 20, repr.plot.height = 8)
p2

#ggsave("./6-kmita/images/E10_5_m_distal.png" ,width=20, height=8)
#ggsave("./6-kmita/images/E10_5_m_distal2.png" ,width=10, height=8, plot=p1)

In [ ]:
sc_mesenchymal_E10_5@meta.data$CellType_PID <- Idents(sc_mesenchymal_E10_5)

In [ ]:
sc_mesenchymal_E10_5@meta.data$CellType_PID <- as.character(sc_mesenchymal_E10_5@meta.data$CellType_PID)

sc_mesenchymal_E10_5@meta.data$CellType_PID[sc_mesenchymal_E10_5@meta.data$CellType_PID == '0'] <- 'Proximal cells'
sc_mesenchymal_E10_5@meta.data$CellType_PID[sc_mesenchymal_E10_5@meta.data$CellType_PID == '10'] <- 'Proximal cells'
sc_mesenchymal_E10_5@meta.data$CellType_PID[sc_mesenchymal_E10_5@meta.data$CellType_PID == '11'] <- 'Proximal cells'
sc_mesenchymal_E10_5@meta.data$CellType_PID[sc_mesenchymal_E10_5@meta.data$CellType_PID == '17'] <- 'Proximal cells'
sc_mesenchymal_E10_5@meta.data$CellType_PID[sc_mesenchymal_E10_5@meta.data$CellType_PID == '20'] <- 'Proximal cells'
sc_mesenchymal_E10_5@meta.data$CellType_PID[sc_mesenchymal_E10_5@meta.data$CellType_PID == '21'] <- 'Proximal cells'
sc_mesenchymal_E10_5@meta.data$CellType_PID[sc_mesenchymal_E10_5@meta.data$CellType_PID == '24'] <- 'Proximal cells'
sc_mesenchymal_E10_5@meta.data$CellType_PID[sc_mesenchymal_E10_5@meta.data$CellType_PID == '7'] <- 'Proximal cells'
sc_mesenchymal_E10_5@meta.data$CellType_PID[sc_mesenchymal_E10_5@meta.data$CellType_PID == '8'] <- 'Proximal cells'
sc_mesenchymal_E10_5@meta.data$CellType_PID[sc_mesenchymal_E10_5@meta.data$CellType_PID == '12'] <- 'Proximal cells'

sc_mesenchymal_E10_5@meta.data$CellType_PID[sc_mesenchymal_E10_5@meta.data$CellType_PID == '2'] <- 'Intermediate cells'
sc_mesenchymal_E10_5@meta.data$CellType_PID[sc_mesenchymal_E10_5@meta.data$CellType_PID == '13'] <- 'Intermediate cells'
sc_mesenchymal_E10_5@meta.data$CellType_PID[sc_mesenchymal_E10_5@meta.data$CellType_PID == '23'] <- 'Intermediate cells'
sc_mesenchymal_E10_5@meta.data$CellType_PID[sc_mesenchymal_E10_5@meta.data$CellType_PID == '4'] <- 'Intermediate cells'
sc_mesenchymal_E10_5@meta.data$CellType_PID[sc_mesenchymal_E10_5@meta.data$CellType_PID == '5'] <- 'Intermediate cells'
sc_mesenchymal_E10_5@meta.data$CellType_PID[sc_mesenchymal_E10_5@meta.data$CellType_PID == '9'] <- 'Intermediate cells'
sc_mesenchymal_E10_5@meta.data$CellType_PID[sc_mesenchymal_E10_5@meta.data$CellType_PID == '14'] <- 'Intermediate cells'
sc_mesenchymal_E10_5@meta.data$CellType_PID[sc_mesenchymal_E10_5@meta.data$CellType_PID == '18'] <- 'Intermediate cells'
sc_mesenchymal_E10_5@meta.data$CellType_PID[sc_mesenchymal_E10_5@meta.data$CellType_PID == '1'] <- 'Intermediate cells'
sc_mesenchymal_E10_5@meta.data$CellType_PID[sc_mesenchymal_E10_5@meta.data$CellType_PID == '25'] <- 'Intermediate cells'
sc_mesenchymal_E10_5@meta.data$CellType_PID[sc_mesenchymal_E10_5@meta.data$CellType_PID == '3'] <- 'Intermediate cells'

sc_mesenchymal_E10_5@meta.data$CellType_PID[sc_mesenchymal_E10_5@meta.data$CellType_PID == '6'] <- 'Distal cells'
sc_mesenchymal_E10_5@meta.data$CellType_PID[sc_mesenchymal_E10_5@meta.data$CellType_PID == '15'] <- 'Distal cells'
sc_mesenchymal_E10_5@meta.data$CellType_PID[sc_mesenchymal_E10_5@meta.data$CellType_PID == '16'] <- 'Distal cells'
sc_mesenchymal_E10_5@meta.data$CellType_PID[sc_mesenchymal_E10_5@meta.data$CellType_PID == '19'] <- 'Distal cells'
sc_mesenchymal_E10_5@meta.data$CellType_PID[sc_mesenchymal_E10_5@meta.data$CellType_PID == '22'] <- 'Distal cells'


sc_mesenchymal_E10_5@meta.data$CellType_PID <- as.factor(sc_mesenchymal_E10_5@meta.data$CellType_PID)

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 8)

p1 <- DimPlot(sc_mesenchymal_E10_5, group.by = 'CellType_PID') + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 9.5, repr.plot.height = 8)
p1 

**Cell type**


ZPA

In [ ]:
g <- WhichCells(sc_mesenchymal_E10_5, idents = c("16"))
p1 <- DimPlot(sc_mesenchymal_E10_5, cells.highlight = list(g)) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E10_5, c("Shh")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 20, repr.plot.height = 8)
p1 + p2

#ggsave("./6-kmita/images/E10_5_m_ZPA.png" ,width=20, height=8)

Chondrocyte precursors

In [ ]:
g <- WhichCells(sc_mesenchymal_E10_5, idents = c("11"))
p1 <- DimPlot(sc_mesenchymal_E10_5, cells.highlight = list(g)) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E10_5, c("Sox9", "Prrx1","Prrx2")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 9.5, repr.plot.height = 8)
p1 
options(repr.plot.width = 20, repr.plot.height = 16)
p2

In [ ]:
g <- WhichCells(sc_mesenchymal_E10_5, idents = c('8','7','5',"20"))
p1 <- DimPlot(sc_mesenchymal_E10_5, cells.highlight = list(g), label = TRUE) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E10_5, c("Sox9")) + xlab("UMAP 1") + ylab("UMAP 2") 

p2 <- FeaturePlot(sc_mesenchymal_E10_5, c("Sox9", "Prrx1","Prrx2")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 9.5, repr.plot.height = 8)
p1 
options(repr.plot.width = 20, repr.plot.height = 16)
p2

In [ ]:
sc_mesenchymal_E10_5@meta.data$CellType_g <- Idents(sc_mesenchymal_E10_5)

In [ ]:
sc_mesenchymal_E10_5@meta.data$CellType_g <- as.character(sc_mesenchymal_E10_5@meta.data$CellType_g)

sc_mesenchymal_E10_5@meta.data$CellType_g[sc_mesenchymal_E10_5@meta.data$CellType_g == '8'] <- 'Chondrocyte precursors'
sc_mesenchymal_E10_5@meta.data$CellType_g[sc_mesenchymal_E10_5@meta.data$CellType_g == '7'] <- 'Chondrocyte precursors'
sc_mesenchymal_E10_5@meta.data$CellType_g[sc_mesenchymal_E10_5@meta.data$CellType_g == '5'] <- 'Chondrocyte precursors'
sc_mesenchymal_E10_5@meta.data$CellType_g[sc_mesenchymal_E10_5@meta.data$CellType_g == '20'] <- 'Chondrocyte precursors'

sc_mesenchymal_E10_5@meta.data$CellType_g[sc_mesenchymal_E10_5@meta.data$CellType_g == '16'] <- 'ZPA'

sc_mesenchymal_E10_5@meta.data$CellType_g[sc_mesenchymal_E10_5@meta.data$CellType_g == '0'] <- 'Mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType_g[sc_mesenchymal_E10_5@meta.data$CellType_g == '1'] <- 'Mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType_g[sc_mesenchymal_E10_5@meta.data$CellType_g == '10'] <- 'Mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType_g[sc_mesenchymal_E10_5@meta.data$CellType_g == '11'] <- 'Mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType_g[sc_mesenchymal_E10_5@meta.data$CellType_g == '12'] <- 'Mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType_g[sc_mesenchymal_E10_5@meta.data$CellType_g == '13'] <- 'Mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType_g[sc_mesenchymal_E10_5@meta.data$CellType_g == '14'] <- 'Mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType_g[sc_mesenchymal_E10_5@meta.data$CellType_g == '15'] <- 'Mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType_g[sc_mesenchymal_E10_5@meta.data$CellType_g == '17'] <- 'Mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType_g[sc_mesenchymal_E10_5@meta.data$CellType_g == '18'] <- 'Mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType_g[sc_mesenchymal_E10_5@meta.data$CellType_g == '19'] <- 'Mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType_g[sc_mesenchymal_E10_5@meta.data$CellType_g == '2'] <- 'Mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType_g[sc_mesenchymal_E10_5@meta.data$CellType_g == '21'] <- 'Mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType_g[sc_mesenchymal_E10_5@meta.data$CellType_g == '22'] <- 'Mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType_g[sc_mesenchymal_E10_5@meta.data$CellType_g == '23'] <- 'Mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType_g[sc_mesenchymal_E10_5@meta.data$CellType_g == '24'] <- 'Mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType_g[sc_mesenchymal_E10_5@meta.data$CellType_g == '25'] <- 'Mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType_g[sc_mesenchymal_E10_5@meta.data$CellType_g == '3'] <- 'Mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType_g[sc_mesenchymal_E10_5@meta.data$CellType_g == '4'] <- 'Mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType_g[sc_mesenchymal_E10_5@meta.data$CellType_g == '6'] <- 'Mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType_g[sc_mesenchymal_E10_5@meta.data$CellType_g == '9'] <- 'Mesenchyme'

sc_mesenchymal_E10_5@meta.data$CellType_g <- as.factor(sc_mesenchymal_E10_5@meta.data$CellType_g)

In [ ]:
p1 <- DimPlot(sc_mesenchymal_E10_5, group.by = 'CellType_g') + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E10_5, c("Shh", "Sox9")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 10, repr.plot.height = 8)
p1 
options(repr.plot.width = 20, repr.plot.height = 8)
p2

**Combining positional information with cell types**

In [ ]:
sc_mesenchymal_E10_5@meta.data$CellType <- Idents(sc_mesenchymal_E10_5)

In [ ]:
sc_mesenchymal_E10_5@meta.data$CellType <- as.character(sc_mesenchymal_E10_5@meta.data$CellType)

sc_mesenchymal_E10_5@meta.data$CellType[sc_mesenchymal_E10_5@meta.data$CellType == '16'] <- 'ZPA'

sc_mesenchymal_E10_5@meta.data$CellType[sc_mesenchymal_E10_5@meta.data$CellType == '0'] <- 'Proximal limb mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType[sc_mesenchymal_E10_5@meta.data$CellType == '10'] <- 'Proximal limb mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType[sc_mesenchymal_E10_5@meta.data$CellType == '11'] <- 'Proximal limb mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType[sc_mesenchymal_E10_5@meta.data$CellType == '12'] <- 'Proximal limb mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType[sc_mesenchymal_E10_5@meta.data$CellType == '17'] <- 'Proximal limb mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType[sc_mesenchymal_E10_5@meta.data$CellType == '21'] <- 'Proximal limb mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType[sc_mesenchymal_E10_5@meta.data$CellType == '24'] <- 'Proximal limb mesenchyme'

sc_mesenchymal_E10_5@meta.data$CellType[sc_mesenchymal_E10_5@meta.data$CellType == '1'] <- 'Intermediate limb mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType[sc_mesenchymal_E10_5@meta.data$CellType == '13'] <- 'Intermediate limb mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType[sc_mesenchymal_E10_5@meta.data$CellType == '14'] <- 'Intermediate limb mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType[sc_mesenchymal_E10_5@meta.data$CellType == '18'] <- 'Intermediate limb mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType[sc_mesenchymal_E10_5@meta.data$CellType == '2'] <- 'Intermediate limb mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType[sc_mesenchymal_E10_5@meta.data$CellType == '23'] <- 'Intermediate limb mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType[sc_mesenchymal_E10_5@meta.data$CellType == '25'] <- 'Intermediate limb mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType[sc_mesenchymal_E10_5@meta.data$CellType == '3'] <- 'Intermediate limb mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType[sc_mesenchymal_E10_5@meta.data$CellType == '4'] <- 'Intermediate limb mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType[sc_mesenchymal_E10_5@meta.data$CellType == '9'] <- 'Intermediate limb mesenchyme'

sc_mesenchymal_E10_5@meta.data$CellType[sc_mesenchymal_E10_5@meta.data$CellType == '15'] <- 'Distal limb mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType[sc_mesenchymal_E10_5@meta.data$CellType == '19'] <- 'Distal limb mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType[sc_mesenchymal_E10_5@meta.data$CellType == '22'] <- 'Distal limb mesenchyme'
sc_mesenchymal_E10_5@meta.data$CellType[sc_mesenchymal_E10_5@meta.data$CellType == '6'] <- 'Distal limb mesenchyme'

sc_mesenchymal_E10_5@meta.data$CellType[sc_mesenchymal_E10_5@meta.data$CellType == '20'] <- 'Chondroprogenitors'
sc_mesenchymal_E10_5@meta.data$CellType[sc_mesenchymal_E10_5@meta.data$CellType == '5'] <- 'Chondroprogenitors'
sc_mesenchymal_E10_5@meta.data$CellType[sc_mesenchymal_E10_5@meta.data$CellType == '7'] <- 'Chondroprogenitors'
sc_mesenchymal_E10_5@meta.data$CellType[sc_mesenchymal_E10_5@meta.data$CellType == '8'] <- 'Chondroprogenitors'

sc_mesenchymal_E10_5@meta.data$CellType <- as.factor(sc_mesenchymal_E10_5@meta.data$CellType)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)
DimPlot(sc_mesenchymal_E10_5, group.by = 'CellType') + xlab("UMAP 1") + ylab("UMAP 2")

In [ ]:
saveRDS(sc_mesenchymal_E10_5, file = "/home/host_home/tom/Kmita/6-kmita/RDataSessions/sc_mesenchymal_E10_5_annotated_SeuratObject.Rds")

##### 3.4.2.2. Subsetting ectodermal cluster<a id="15"></a>

In [ ]:
E10_5 <- readRDS("/home/host_home/tom/Kmita/6-kmita/RDataSessions/E10_5_GlobalAnnotation_SeuratObject.Rds")

In [ ]:
Idents(E10_5) <- E10_5@meta.data$CellType

In [ ]:
sc_ectoderm_E10_5 <- subset(E10_5, idents = c("Ectoderm"))

In [ ]:
sc_ectoderm_E10_5 <- NormalizeData(sc_ectoderm_E10_5, verbose = FALSE)
sc_ectoderm_E10_5 <- FindVariableFeatures(sc_ectoderm_E10_5, selection.method = "vst", nfeatures = 2000)

# Identify the 10 most highly variable genes
top10 <- head(VariableFeatures(sc_ectoderm_E10_5), 10)

In [ ]:
sc_ectoderm_E10_5 <- ScaleData(sc_ectoderm_E10_5)

In [ ]:
sc_ectoderm_E10_5 <- RunPCA(sc_ectoderm_E10_5, features = VariableFeatures(object = sc_ectoderm_E10_5), verbose = FALSE)

In [ ]:
options(repr.plot.width = 9)

DimPlot(sc_ectoderm_E10_5, reduction = "pca", group.by = "orig.ident") + xlab("PC 1") + ylab("PC 2")

In [ ]:
ElbowPlot(sc_ectoderm_E10_5, 50)

In [ ]:
sc_ectoderm_E10_5 <- FindNeighbors(sc_ectoderm_E10_5, dims = 1:20)
sc_ectoderm_E10_5 <- FindClusters(sc_ectoderm_E10_5, resolution = 0.2)

In [ ]:
sc_ectoderm_E10_5 <- RunUMAP(sc_ectoderm_E10_5, dims = 1:20, n.neighbors = 10, verbose = FALSE)

In [ ]:
options(repr.plot.width=8)
DimPlot(sc_ectoderm_E10_5, reduction = "umap", label=TRUE, group.by = "seurat_clusters")

#ggsave("./6-kmita/images/E10_5_e_no_annotation.png" ,width=8, height=8)

In [ ]:
df <- FindAllMarkers(sc_ectoderm_E10_5)

In [ ]:
#write.csv(df, file = "./6-kmita/RDataSessions/E10_5_GlobalAnnotation_SeuratObject.Rds", quote = FALSE)

In [ ]:
# find markers for every cluster compared to all remaining cells, report only the positive
# ones
#df %>%
#    group_by(cluster) %>%
#    slice_max(n = 10, order_by = avg_log2FC) 

In [ ]:
#select only cluster 0 and order avg_log2FC from big to small

df[df$cluster == 4,] %>%
slice_max(n = 50, order_by = avg_log2FC)

In [ ]:
#df['En1',]

Subcluster: AER

In [ ]:
options(repr.plot.width = 20)

p1 <- DimPlot(sc_ectoderm_E10_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_ectoderm_E10_5, c("Fgf8")) + xlab("UMAP 1") + ylab("UMAP 2")
p3 <- FeaturePlot(sc_ectoderm_E10_5, c("Rspo2")) + xlab("UMAP 1") + ylab("UMAP 2")

p4 <- p3 + p2
p5 <- p1 + p4
p5
#ggsave("./6-kmita/images/E10_5_e_annotated.png" ,width=20, height=8)
#remove "Bmp4", "Dlx5"

In [ ]:
options(repr.plot.width = 20)

p1 <- DimPlot(sc_ectoderm_E10_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_ectoderm_E10_5, c("En1", "Wnt7a","Fgfr2","Lrp4")) + xlab("UMAP 1") + ylab("UMAP 2")
p3 <- FeaturePlot(sc_ectoderm_E10_5, c("Wnt7a")) + xlab("UMAP 1") + ylab("UMAP 2")

p1 + p2

In [ ]:
options(repr.plot.width = 20)

p1 <- DimPlot(sc_ectoderm_E10_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_ectoderm_E10_5, c("Krt5", "Krt10")) + xlab("UMAP 1") + ylab("UMAP 2")

p1 + p2

In [ ]:
options(repr.plot.width=8)
DimPlot(sc_ectoderm_E10_5, reduction = "umap", label=TRUE, group.by = "seurat_clusters")

In [ ]:
sc_ectoderm_E10_5@meta.data$CellType <- Idents(sc_ectoderm_E10_5)

In [ ]:
sc_ectoderm_E10_5@meta.data$CellType <- as.character(sc_ectoderm_E10_5@meta.data$CellType)

sc_ectoderm_E10_5@meta.data$CellType[sc_ectoderm_E10_5@meta.data$CellType == '0'] <- 'Ectoderm'
sc_ectoderm_E10_5@meta.data$CellType[sc_ectoderm_E10_5@meta.data$CellType == '1'] <- 'AER'
sc_ectoderm_E10_5@meta.data$CellType[sc_ectoderm_E10_5@meta.data$CellType == '2'] <- 'Skin'

sc_ectoderm_E10_5@meta.data$CellType <- as.factor(sc_ectoderm_E10_5@meta.data$CellType)

In [ ]:
DimPlot(sc_ectoderm_E10_5, group.by = 'CellType') + xlab("UMAP 1") + ylab("UMAP 2")

In [ ]:
saveRDS(sc_ectoderm_E10_5, file = "/home/host_home/tom/Kmita/6-kmita/RDataSessions/sc_ectoderm_E10_5_annotated_SeuratObject.Rds")

#### 3.4.2. Updating annotations<a id="16"></a>

In [ ]:
E10_5 <- readRDS("/home/host_home/tom/Kmita/6-kmita/RDataSessions/E10_5_GlobalAnnotation_SeuratObject.Rds")

In [ ]:
sc_mesenchymal_E10_5 <- readRDS(file = "/home/host_home/tom/Kmita/6-kmita/RDataSessions/sc_mesenchymal_E10_5_annotated_SeuratObject.Rds")

In [ ]:
sc_ectoderm_E10_5 <- readRDS(file = "/home/host_home/tom/Kmita/6-kmita/RDataSessions/sc_ectoderm_E10_5_annotated_SeuratObject.Rds")

In [ ]:
meta1 <- E10_5@meta.data
meta2 <- sc_mesenchymal_E10_5@meta.data

In [ ]:
df <- split(meta1, meta1$CellType)

In [ ]:
mesenchyme <- rownames(df$Mesenchyme)

In [ ]:
mesenchyme_meta <- meta2[mesenchyme, c("CellType", "orig.ident")]
mesenchyme_meta$orig.ident <- NULL

head(mesenchyme_meta)

In [ ]:
df$Mesenchyme <- cbind(df$Mesenchyme, mesenchyme_meta)
df$Mesenchyme$CellType <- df$Mesenchyme$CellType
df$Mesenchyme$CellType <- NULL

In [ ]:
str(df)

In [ ]:
ectoderm <- rownames(df$Ectoderm)

In [ ]:
meta2 <- sc_ectoderm_E10_5@meta.data
ectoderm_meta <- meta2[ectoderm, c("CellType", "orig.ident")]
ectoderm_meta$orig.ident <- NULL

head(ectoderm_meta)

In [ ]:
df$Ectoderm <- cbind(df$Ectoderm, ectoderm_meta)
df$Ectoderm$CellType <- df$Ectoderm$CellType

In [ ]:
df$Mesenchyme <- cbind(df$Mesenchyme, mesenchyme_meta)
df$Mesenchyme$CellType <- df$Mesenchyme$CellType

In [ ]:
df$Mesenchyme$Barcode = rownames(df$Mesenchyme)
df$Ectoderm$Barcode = rownames(df$Ectoderm)

**Make sure all dataframes in the list have the same number of columns, in the same order.**

In [ ]:
str(df)

In [ ]:
df$Mesenchyme$CellType <- NULL
df$Ectoderm$CellType <- NULL

In [ ]:
str(df)

In [ ]:
names(df)

In [ ]:
df$'Vascular endothelial cells'$Barcode <- rownames(df$'Vascular endothelial cells')
df$'Erythroid cells'$Barcode <- rownames(df$'Erythroid cells')
df$'Muscle progenitors'$Barcode <- rownames(df$'Muscle progenitors')

In [ ]:
str(df)

In [ ]:
meta.data <- do.call(rbind, df)
rownames(meta.data) = meta.data$Barcode
meta.data$Barcode <- NULL

In [ ]:
head(meta.data)

In [ ]:
dim(E10_5@meta.data)
dim(meta.data)

In [ ]:
target <- rownames(E10_5@meta.data)
meta.data <- meta.data[match(target, rownames(meta.data)),]

In [ ]:
E10_5@meta.data = meta.data

#### 5.4.3. Finalized embedding and annotation<a id="37"></a>

In [ ]:
options(repr.plot.width=12)
DimPlot(E10_5, group.by = "CellType", label = FALSE, repel = TRUE) + xlab("UMAP 1") + ylab("UMAP 2")

#ggsave("./6-kmita/images/E10_5_annotated.png" ,width=12, height=8)

In [ ]:
Idents(E10_5) <- E10_5$CellType

In [ ]:
saveRDS(E10_5, file = "/home/host_home/tom/Kmita/6-kmita/RDataSessions/E10_5_seuratObject_annotation.Rds")

In [ ]:
E10_5